In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import random
random.seed(123)

import json
import os

%run Code/plot_setup.py

%reload_ext autoreload
%autoreload 2

cwd = os.getcwd()

In [2]:
os.chdir(cwd + "/Investment_Funnel_SOM")
from models.main import TradeBot

# setup funnel
algo = TradeBot()

In [3]:
# setup data

# full set
start = "2015-01-15"
end = "2022-11-24"

train_test_datesplit = "2019-01-01"

# SETUP WORKING DATASET, DIVIDE DATASET INTO TRAINING AND TESTING PART?
algo.setup_data(
    start=start, 
    end=end, 
    train_test=True,  
    end_train = train_test_datesplit, 
    start_test = train_test_datesplit
)


In [4]:
# etf symbols and names
with open("../Data/Data_ETFs_Names.json", "r") as openfile:
    # Reading from json file
    etf_dict = json.load(openfile)
    
# benchmarks
with open("../Data/Benchmarks.json", "r") as openfile:
    # Reading from json file
    nord_benchmarks = json.load(openfile)
    
risk_level_weights = {}
for key,  item in nord_benchmarks.items():
    risk_level_weights[key] = [round(w, 4) for w in item.get("weights")]
    
    
# subsets
with open("../Data/Data_ETFs_Subsets(Large).json", "r") as openfile:
    # Reading from json file
    portfolio_subsets = json.load(openfile)
    
nord_subset = etf_dict["symbols"].get("nord")
universe_subset = portfolio_subsets.pop("Universe")


# for port in ['SOM-Cluster', 'Cluster', "SOM-MST", "MST"]:
#     tmp_pop = portfolio_subsets.pop(port)
# 
# for port in ['nord_13', 'nord_20']:
#     tmp_pop = risk_level_weights.pop(port)
    
for portfolio, sub in portfolio_subsets.items():
    print(f"n = {len(sub)} in {portfolio}")

n = 40 in SOM-Cluster
n = 42 in Cluster
n = 57 in SOM-MST
n = 43 in MST


In [5]:
from models.CVaRtargets import get_cvar_targets

# get cvar targets
# benchmark_names = ['iShares MSCI World ETF']
# benchmark_symbols = [algo.tickers[list(algo.names).index(name)] for name in benchmark_names]

# Number of scenarios
n_simulations = 1000
n_rebalancing = 12

benchmarks = {}
for level, level_weights in risk_level_weights.items():

    print(f"\nGetting targets for: {level}")
    benchmark_symbols = nord_subset
    benchmark_weights = level_weights

    targets, benchmark_port_val = get_cvar_targets(
        start_date = algo.start,
        end_date = algo.end,
        test_date = algo.startTestDate,
        benchmark = benchmark_symbols,  # MSCI World benchmark
        weights = benchmark_weights, # benchmark weights
        test_index = algo.testDataset.index.date,
        budget = 100,
        cvar_alpha = 0.05,
        data = algo.weeklyReturns,
        n_simulations = n_simulations,
        n_rebalancing = n_rebalancing
    )
    
    # benchmark_port_val.columns = [level]
    # benchmarks[level] = {"targets": targets, "value": benchmark_port_val}

    benchmarks[level] = {
        "targets": targets["CVaR_Target"].to_list(), 
        "value": benchmark_port_val["Benchmark_Value"].to_list(),
    }


Getting targets for: nord_5
Scenario generation: Monte Carlo

-------Simulating Weekly Returns-------
-------Computing Period Returns-------

Getting targets for: nord_13
Scenario generation: Monte Carlo

-------Simulating Weekly Returns-------
-------Computing Period Returns-------

Getting targets for: nord_20
Scenario generation: Monte Carlo

-------Simulating Weekly Returns-------
-------Computing Period Returns-------


In [6]:
from models.CVaRmodel import cvar_model
from models.dataAnalyser import final_stats
from models.ScenarioGeneration import bootstrapping, monte_carlo

portfolio_data = {}

for level, values in benchmarks.items():
    
    portfolio_data[level] = {}
    print(f"Optimising portfolio allocation. (targets = {level})\n")
    for subset_name, subset in portfolio_subsets.items():

        print(f"\tOptimising allocation of: {subset_name}")
        portfolio_data[level][subset_name] = {}

        scenarios = monte_carlo(
            data = algo.weeklyReturns[subset],  # subsetMST or subsetCLUST
            n_simulations = n_simulations,  # number of scenarios per period
            n_test = algo.lenTest,
            n_rebalancing = n_rebalancing
        )

        # get optimal portfolio allocation
        port_allocation, port_value, port_cvar = cvar_model(
            test_ret = algo.testDataset[subset],
            scenarios = scenarios,  # Scenarios
            targets = benchmarks[level]["targets"],  # Target
            budget = 100,
            cvar_alpha = 0.05,
            trans_cost = 0.001,
            max_weight = 1,
            n_rebalancing = n_rebalancing
        )


        portfolio_data[level][subset_name].update({"allocation":port_allocation.to_dict()})
        portfolio_data[level][subset_name].update({"value":port_value["Portfolio_Value"].to_list()})
        portfolio_data[level][subset_name].update({"cvar":port_cvar["CVaR"].to_list()})

    print(f"\nFinished.\n")

Optimising portfolio allocation. (targets = nord_5)

	Optimising allocation of: SOM-Cluster
-------Simulating Weekly Returns-------
-------Computing Monthly Returns-------
	Optimising allocation of: Cluster
-------Simulating Weekly Returns-------
-------Computing Monthly Returns-------
	Optimising allocation of: SOM-MST
-------Simulating Weekly Returns-------
-------Computing Monthly Returns-------
	Optimising allocation of: MST
-------Simulating Weekly Returns-------
-------Computing Monthly Returns-------

Finished.

Optimising portfolio allocation. (targets = nord_13)

	Optimising allocation of: SOM-Cluster
-------Simulating Weekly Returns-------
-------Computing Monthly Returns-------
	Optimising allocation of: Cluster
-------Simulating Weekly Returns-------
-------Computing Monthly Returns-------
	Optimising allocation of: SOM-MST
-------Simulating Weekly Returns-------
-------Computing Monthly Returns-------
	Optimising allocation of: MST
-------Simulating Weekly Returns-------
-

In [7]:
date_index = [str(d) for d in benchmark_port_val.index.to_list()]
optimization_data = {
    "benchmarks": benchmarks, 
    "portfolios": portfolio_data,
    "date_index": date_index
}

In [9]:
# # to json
# 
# os.chdir(cwd)
# with open("Data/Optimization_Results(Large).json", "w") as outfile:
#     # Write to json file
#     json.dump(optimization_data, outfile)